# Kaggle Submission Step by Step


** Environment Setup **
load with kaggle cli

pip install kaggle-cli




** Config kaggle cli **
kg config -g -u 'username' -p 'password' -c 'Competettion Name'


In [2]:
# import stuff 
%matplotlib inline
path="data/redux/"
import utils; reload(utils)
from utils import *


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
batch_size=64
valid_size=2000
sample_size=200


In [4]:
# % is for magic commands 

# ! in notebook for the bash commands

import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/redux'


In [5]:
# create a directory structure

%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown


/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux


In [7]:
# in case where you dont have the proper directory structure and also to havee
# a small data set called sample to work with and also valid data set


#get all jpeg image names from train directory

%cd $DATA_HOME_DIR/train
g=glob('*.jpg')

shuf=np.random.permutation(g)

for i in range (valid_size): os.rename(shuf[i],'../valid/'+shuf[i])

/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/train


In [9]:
from shutil import copyfile

In [10]:
# create sample data set and copy files to the new folder structure of sample

g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/' + shuf[i])
    

In [11]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

In [12]:
# Keras needs class wise data in each directory
%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/sample/train
/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/sample/valid
/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/valid
/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/train


In [13]:



# For TEST Data create single 'unknown' class
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/



/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/test


# Modeling and Finetuning


In [14]:
%cd $DATA_HOME_DIR

path=DATA_HOME_DIR+ '/' # change here to test it with sample data or train data

train_path=DATA_HOME_DIR+'/train/'
valid_path=DATA_HOME_DIR+'/valid/'
test_path=DATA_HOME_DIR+'/test/'

results_path=DATA_HOME_DIR+'/results/'


/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux


In [15]:
# VGG Model from the tutorial 1

vgg=Vgg16()

In [16]:
batch_size=64
no_of_epochs=5

In [17]:
batches=vgg.get_batches(train_path,batch_size=batch_size)
val_batches=vgg.get_batches(valid_path,batch_size=2*batch_size)


# now fine tune as we have only 2 classes at the end

# linear classifier/ logistic at the end layer

vgg.finetune(batches)

# now set initially learning reate to be 0.01 that needs to be changed once
# get some satifying results and need to slow down the rate

vgg.model.optimizer.lr=0.01








Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# now save weights at each step so that it is easy to use them next time instead of 
# runing the model again and again

new_weights_filename=None

for aepoch in range (no_of_epochs):
    print "Running EPOCH: %d " % aepoch
    
    vgg.fit(batches,val_batches,nb_epoch=1)
    
    new_weights_filename='ft%d.h5' %aepoch
    vgg.model.save_weights(results_path+new_weights_filename)
    

print "Learning Complete %s fit operations" % no_of_epochs






Running EPOCH: 0 
Epoch 1/1
23000/23000 [==============================] - 607s - loss: 0.3570 - acc: 0.9700 - val_loss: 0.3055 - val_acc: 0.9785
Running EPOCH: 1 
Epoch 1/1
23000/23000 [==============================] - 615s - loss: 0.3198 - acc: 0.9764 - val_loss: 0.3194 - val_acc: 0.9780
Running EPOCH: 2 
Epoch 1/1
23000/23000 [==============================] - 615s - loss: 0.3379 - acc: 0.9766 - val_loss: 0.3564 - val_acc: 0.9750
Running EPOCH: 3 
Epoch 1/1
21400/23000 [==========================>...] - ETA: 39s - loss: 0.3365 - acc: 0.9767

# Prediction on dataset


In [ ]:
batches, predictions =vgg.test(test+path,batch_size=batch_size*2)



In [ ]:
# test function will create 2 classes based on our ordering of the directories

# in this case cat is fist column

print predictions[:5]
filenames=batches.filenames

print filenames[:5]

# to verify the column ordering the values of test predictions

from PIL import 


In [ ]:
# saving test arrays as we have use them for submission
# using cobolz library fast storage of arrays


save_array(results_path+'test_pred.dat', predictions)
save_array(results_path+'filenames.dat',filenames)



# Validating Predictions:



In [ ]:
# load weights 
vgg.model.load_weights(results_path
                      +new_weights_filename)

val_batches, probs=vgg.test(valid_path,batch_size=batch_size)




In [ ]:
# get labels 

filenames=val_batches.filenames
expected_lables=val_batches.classes

# rounding out the 0/1 to generate labels 

our_predictions=probs[:0]
our_labels=np.round(1-our_predictions)





In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(expected_labels, our_labels)

In [ ]:
plot_confusion_matrix(cm,val_batches.class_indices)

# How to submit to Kaggle:

In [ ]:
preds=load_array(results_path+'test_pred.dat')
filenames=load_array(results+path+'filenames.dat')

In [ ]:
#Swap all ones with .95 and all zeros with .05
isdog = preds[:,1]
isdog = isdog.clip(min=0.05, max=0.95)

In [ ]:

#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
# NP Stack will arrange columns

subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_HOME_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')


In [ ]:
# save file using Ipython in chrome 
# we can download this and submit to kaggle
from IPython.display import FileLink
%cd $LESSON_HOME_DIR
FileLink('data/redux/'+submission_file_name)



